In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from quick_pp.objects import Project

# Load well from saved file
project = "Mock"
project_path = rf"data\04_project\{project}.qppp"
project = Project().load(project_path)
project.get_well_names()

# Quick PP Interpretation

In [ ]:
from quick_pp.lithology.sand_silt_clay import SandSiltClay
from quick_pp.porosity import neu_den_xplot_poro, density_porosity, rho_matrix
from quick_pp.qaqc import badhole_flag, mask_outside_threshold, neu_den_xplot_hc_correction
from quick_pp.saturation import *
from quick_pp.plotter import *
from quick_pp.permeability import *
from quick_pp.ressum import *
from quick_pp.rock_type import rock_flagging
from quick_pp.plotter import plotly_log

In [ ]:
df = project.get_all_data()
for well_name, well_data in df.groupby('WELL_NAME'):
    # Load well object
    well = project.get_well(well_name)

    # Clean up data
    well_data = badhole_flag(well_data)
    well_data = mask_outside_threshold(well_data, fill=True)

    # Initialize lithology model
    args = {
        # 'dry_clay_point': (0.27, 2.8)
        'silt_line_angle': 117,
        # 'wet_clay_point': (0.45, 2.45),
    }
    ssc_model = SandSiltClay(**args)
    vsand, vsilt, vcld, vclb = ssc_model.estimate_lithology(
        well_data['NPHI'], well_data['RHOB'], model='kuttan_modified', normalize= True
    )
    args.update(ssc_model.__dict__)
    well.update_config(args)  # Save lithology model to well

    # Implement hydrocarbon correction
    nphihc, rhobhc, hc_flag = neu_den_xplot_hc_correction(
        well_data['NPHI'], well_data['RHOB'], gr=well_data['GR'],
        dry_sand_point=args['dry_sand_point'],
        dry_clay_point=args['dry_clay_point'],
        corr_angle=50
    )

    # Estimate lithology
    ssc_model = SandSiltClay(**args)
    vsand, vsilt, vcld, vclb = ssc_model.estimate_lithology(
        nphihc, rhobhc, model='kuttan_modified', normalize= True
    )
    vclw = vcld + vclb

    # Estimate porosity
    phit = neu_den_xplot_poro(
        nphihc, rhobhc, model='ssc', reservoir=True,
        dry_sand_point=args['dry_sand_point'],
        dry_silt_point=args['dry_silt_point'],
        dry_clay_point=args['dry_clay_point'],
    )

    rho_ma = rho_matrix(vsand, vsilt, vcld)
    phid = density_porosity(rhobhc, rho_ma)
    phie = phit - vclb

    # Estimate water saturation
    temp_grad = estimate_temperature_gradient(well_data['DEPTH'] / 7, 'metric')  # Arbitrarily convert MD to TVD
    water_salinity = 60000
    rw = estimate_rw_temperature_salinity(temp_grad, water_salinity)
    b = estimate_b_waxman_smits(temp_grad, rw)
    qv = estimate_qv(vcld, phit, cec_clay=.1)
    # TODO: Implement m estimation/ based on zonation
    m = 2

    swt = waxman_smits_saturation(well_data['RT'], rw, phit, qv, b, m)
    swt_a = archie_saturation(well_data['RT'], rw, phit, 1, 2, 2)

    # Estimate permeability
    constant = vclb**1.75
    Swirr = constant / phit
    perm = choo_permeability(vclw, vsilt, phit)
    # perm_kc = kozeny_carman_permeability(phit, Swirr)
    # perm_t = timur_permeability(phit, Swirr)
    # perm_c = coates_permeability(phit, Swirr)
    # perm_tx = tixier_permeability(phit, Swirr)

    # Estimate rock types
    rock_flag = rock_flagging(vclw)

    # Estimate reservoir summary
    zones = well_data['ZONES'] = 'ALL' 
    ressum_df = calc_reservoir_summary(well_data.DEPTH, vclw, phit, swt, perm, zones, cutoffs=[0.3, 0.1, .75])

    # Update data in the project
    well_data['NPHI_HC'] = nphihc
    well_data['RHOB_HC'] = rhobhc
    well_data['HC_FLAG'] = hc_flag
    well_data['VSAND'] = vsand
    well_data['VSILT'] = vsilt
    well_data['VCLD'] = vcld
    well_data['VCLW'] = vclw
    well_data['PHIT'] = phit
    well_data['PHIE'] = phie
    well_data['PHID'] = phid
    well_data['RW'] = rw
    well_data['B'] = b
    well_data['QV'] = qv
    well_data['M'] = m
    well_data['SWT'] = swt
    well_data['PERM'] = perm
    well_data['ROCK_FLAG'] = rock_flag

    # Save the results to the well
    well.update_data(well_data)
    well.update_ressum(ressum_df)
    project.save_well(well)


In [ ]:
# Plot individual results
well_name = 'BA-G064'
well = project.get_well(well_name)

fig = plotly_log(well.data, well.depth_uom)
fig.show()

In [ ]:
project.save()